Predicting Energy Behavior of Prosumers in Estonia - A Project on Minimizing Imbalance Costs by Enefit

In [2]:
# Import modules to be used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Wrangling

In [38]:
# Read two csv files that need to be merged
Client_Data=pd.read_csv('D:/group4/predict-energy-behavior-of-prosumers/data/client.csv')
Train_Data=pd.read_csv('D:/group4/predict-energy-behavior-of-prosumers/data/train.csv')
# Data exploration
Train_Data.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [39]:
Train_Data.describe()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id
count,2.018352e+06,2.018352e+06,2.018352e+06,2.017824e+06,2018352.0,2.018352e+06,2.018352e+06,2.018352e+06
mean,7.297034e+00,5.368261e-01,1.898927e+00,2.748556e+02,0.5,3.218746e+02,1.009176e+06,3.304538e+01
std,4.780990e+00,4.986421e-01,1.081766e+00,9.095024e+02,0.5,1.826343e+02,5.826482e+05,1.959059e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.000000e+00,0.000000e+00,1.000000e+00,3.780000e-01,0.0,1.660000e+02,5.045878e+05,1.600000e+01
50%,7.000000e+00,1.000000e+00,2.000000e+00,3.113300e+01,0.5,3.230000e+02,1.009176e+06,3.300000e+01
75%,1.100000e+01,1.000000e+00,3.000000e+00,1.802062e+02,1.0,4.790000e+02,1.513763e+06,5.000000e+01
max,1.500000e+01,1.000000e+00,3.000000e+00,1.548027e+04,1.0,6.370000e+02,2.018351e+06,6.800000e+01


In [40]:
Train_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int64  
 1   is_business         int64  
 2   product_type        int64  
 3   target              float64
 4   is_consumption      int64  
 5   datetime            object 
 6   data_block_id       int64  
 7   row_id              int64  
 8   prediction_unit_id  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 138.6+ MB


In [41]:
# Drop two columns of data that do not need to change data type (to avoid memory error)
Train_Data_list = Train_Data.columns
Train_Data_list = Train_Data_list.drop(['target','datetime'])
Train_Data_list

Index(['county', 'is_business', 'product_type', 'is_consumption',
       'data_block_id', 'row_id', 'prediction_unit_id'],
      dtype='object')

In [42]:
# Change data types of int64 -> int32, and float64 -> float32
Train_Data[Train_Data_list]=Train_Data[Train_Data_list].astype('int32')
Train_Data['target']=Train_Data['target'].astype('float32')
Train_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int32  
 1   is_business         int32  
 2   product_type        int32  
 3   target              float32
 4   is_consumption      int32  
 5   datetime            object 
 6   data_block_id       int32  
 7   row_id              int32  
 8   prediction_unit_id  int32  
dtypes: float32(1), int32(7), object(1)
memory usage: 77.0+ MB


In [43]:
# Change data types for the other dataset
Client_Data.info()
Client_Data_list = Client_Data.columns
Client_Data_list = Client_Data_list.drop(['installed_capacity', 'date'])
Client_Data['installed_capacity'] = Client_Data['installed_capacity'].astype('float32')
Client_Data[Client_Data_list] = Client_Data[Client_Data_list].astype('int32')
Client_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41919 entries, 0 to 41918
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_type        41919 non-null  int64  
 1   county              41919 non-null  int64  
 2   eic_count           41919 non-null  int64  
 3   installed_capacity  41919 non-null  float64
 4   is_business         41919 non-null  int64  
 5   date                41919 non-null  object 
 6   data_block_id       41919 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 2.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41919 entries, 0 to 41918
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_type        41919 non-null  int32  
 1   county              41919 non-null  int32  
 2   eic_count           41919 non-null  int32  
 3   installed_capacity  41919 non-n

In [44]:
# Merge two datasets based on common variables (county, whether or not a business, product type, and a unique identifier for each type of prosumer)
Train_Data.info()
Client_Data.info()
merged_client_train = pd.merge(Train_Data, Client_Data, on=['county', 'is_business', 'product_type', 'data_block_id'], how='left')
merged_client_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int32  
 1   is_business         int32  
 2   product_type        int32  
 3   target              float32
 4   is_consumption      int32  
 5   datetime            object 
 6   data_block_id       int32  
 7   row_id              int32  
 8   prediction_unit_id  int32  
dtypes: float32(1), int32(7), object(1)
memory usage: 77.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41919 entries, 0 to 41918
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_type        41919 non-null  int32  
 1   county              41919 non-null  int32  
 2   eic_count           41919 non-null  int32  
 3   installed_capacity  41919 non-null  float32
 4   is_business         41919 non-null  int32  
 5   date  

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,eic_count,installed_capacity,date
0,0,0,1,0.713000,0,2021-09-01 00:00:00,0,0,0,NaN,NaN,NaN
1,0,0,1,96.589996,1,2021-09-01 00:00:00,0,1,0,NaN,NaN,NaN
2,0,0,2,0.000000,0,2021-09-01 00:00:00,0,2,1,NaN,NaN,NaN
3,0,0,2,17.313999,1,2021-09-01 00:00:00,0,3,1,NaN,NaN,NaN
4,0,0,3,2.904000,0,2021-09-01 00:00:00,0,4,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233002,1,2023-05-31 23:00:00,637,2018347,64,15.0,620.000000,2023-05-29
2018348,15,1,1,0.000000,0,2023-05-31 23:00:00,637,2018348,59,20.0,624.500000,2023-05-29
2018349,15,1,1,28.403999,1,2023-05-31 23:00:00,637,2018349,59,20.0,624.500000,2023-05-29
2018350,15,1,3,0.000000,0,2023-05-31 23:00:00,637,2018350,60,55.0,2188.199951,2023-05-29


In [45]:
merged_client_train.isnull().sum()

county                   0
is_business              0
product_type             0
target                 528
is_consumption           0
datetime                 0
data_block_id            0
row_id                   0
prediction_unit_id       0
eic_count             8640
installed_capacity    8640
date                  8640
dtype: int64

In [46]:
# Create a merged new dataset
merged_client_train.to_csv('D:/group4/predict-energy-behavior-of-prosumers/data/client_train.csv',index=False)

In [47]:
# Open the merged dataset
merged_client_train=pd.read_csv('D:/group4/predict-energy-behavior-of-prosumers/data/client_train.csv')

In [48]:
merged_client_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,eic_count,installed_capacity,date
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,15.0,620.0,2023-05-29
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,20.0,624.5,2023-05-29
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,20.0,624.5,2023-05-29
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,55.0,2188.2,2023-05-29


In [49]:
# Open a json file containing county names
CountyName = pd.read_json('D:\group4\predict-energy-behavior-of-prosumers\data\county_id_to_name_map.json',orient='index')
CountyName

,0
0,HARJUMAA
1,HIIUMAA
2,IDA-VIRUMAA
3,JÄRVAMAA
4,JÕGEVAMAA
5,LÄÄNE-VIRUMAA
6,LÄÄNEMAA
7,PÄRNUMAA
8,PÕLVAMAA
9,RAPLAMAA


In [50]:
# Create a column of index for all counties
CountyName['county'] = CountyName.index
CountyName['county_name'] = CountyName[0]
CountyName=CountyName[['county','county_name']]
CountyName

,county,county_name
0,0,HARJUMAA
1,1,HIIUMAA
2,2,IDA-VIRUMAA
3,3,JÄRVAMAA
4,4,JÕGEVAMAA
5,5,LÄÄNE-VIRUMAA
6,6,LÄÄNEMAA
7,7,PÄRNUMAA
8,8,PÕLVAMAA
9,9,RAPLAMAA


In [51]:
# Merge county names with corresponding county numbers based on the index
merged_client_train_count = pd.merge(merged_client_train,CountyName, on=['county'], how = 'left')
merged_client_train_count

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,eic_count,installed_capacity,date,county_name
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,NaN,NaN,NaN,HARJUMAA
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,NaN,NaN,NaN,HARJUMAA
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,NaN,NaN,NaN,HARJUMAA
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,NaN,NaN,NaN,HARJUMAA
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,NaN,NaN,NaN,HARJUMAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,15.0,620.0,2023-05-29,VÕRUMAA
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,20.0,624.5,2023-05-29,VÕRUMAA
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,20.0,624.5,2023-05-29,VÕRUMAA
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,55.0,2188.2,2023-05-29,VÕRUMAA


In [52]:
# Create a new csv file with merged county names
merged_client_train_count.to_csv('D:/group4/predict-energy-behavior-of-prosumers/data/merged_client_train_count.csv',index=False)